In [1]:
import chromadb
import pandas as pd
import csv
import random
from transformers import AutoTokenizer, AutoModel
from chromadb.utils import embedding_functions
import os
chroma_client = chromadb.PersistentClient(
    path=os.getcwd()
)

/Users/jacobzimmerman/anaconda3/envs/project_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer


In [3]:
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
chroma_client.get_or_create_collection(
    name='Collection_01',
    embedding_function=embedding_model
)

/Users/jacobzimmerman/anaconda3/envs/project_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Collection(id=2094b943-e289-49fb-9807-00fb310ec65b, name=Collection_01)

In [4]:
new_collection=chroma_client.get_collection(name='Collection_01')

In [5]:
df = pd.read_csv('imdb_top_1000.csv')
movie_df = df
movie_df = movie_df.drop(['Poster_Link','Certificate','IMDB_Rating','No_of_Votes','Gross'],axis=1)
movie_df['Actors'] = movie_df.apply(lambda row: [row['Star1'],row['Star2'],row['Star3'],row['Star4']],axis=1)

In [6]:
data_df = movie_df[['Series_Title','Overview']]
data_df['full info'] = data_df['Series_Title']+': '+data_df['Overview']

/var/folders/0z/235t747x1r18ndfq4fhdwbvm0000gn/T/ipykernel_15490/110980135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['full info'] = data_df['Series_Title']+': '+data_df['Overview']


In [7]:
metas = movie_df.drop(columns=['Series_Title','Overview'])
metas

,Released_Year,Runtime,Genre,Meta_score,Director,Star1,Star2,Star3,Star4,Actors
0,1994,142 min,Drama,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,1972,175 min,"Crime, Drama",100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,2008,152 min,"Action, Crime, Drama",84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,1974,202 min,"Crime, Drama",90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
4,1957,96 min,"Crime, Drama",96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
...,...,...,...,...,...,...,...,...,...,...
995,1961,115 min,"Comedy, Drama, Romance",76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,"[Audrey Hepburn, George Peppard, Patricia Neal..."
996,1956,201 min,"Drama, Western",84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,"[Elizabeth Taylor, Rock Hudson, James Dean, Ca..."
997,1953,118 min,"Drama, Romance, War",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,"[Burt Lancaster, Montgomery Clift, Deborah Ker..."
998,1944,97 min,"Drama, War",78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,"[Tallulah Bankhead, John Hodiak, Walter Slezak..."


In [8]:
metas=metas.drop(['Star1','Star2','Star3','Star4'],axis=1)
metas

,Released_Year,Runtime,Genre,Meta_score,Director,Actors
0,1994,142 min,Drama,80.0,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,1972,175 min,"Crime, Drama",100.0,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,2008,152 min,"Action, Crime, Drama",84.0,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,1974,202 min,"Crime, Drama",90.0,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
4,1957,96 min,"Crime, Drama",96.0,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
...,...,...,...,...,...,...
995,1961,115 min,"Comedy, Drama, Romance",76.0,Blake Edwards,"[Audrey Hepburn, George Peppard, Patricia Neal..."
996,1956,201 min,"Drama, Western",84.0,George Stevens,"[Elizabeth Taylor, Rock Hudson, James Dean, Ca..."
997,1953,118 min,"Drama, Romance, War",85.0,Fred Zinnemann,"[Burt Lancaster, Montgomery Clift, Deborah Ker..."
998,1944,97 min,"Drama, War",78.0,Alfred Hitchcock,"[Tallulah Bankhead, John Hodiak, Walter Slezak..."


In [9]:
filtered = df[metas['Released_Year']=='PG']
filtered

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
966,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,"Adventure, Drama, History",7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,Tom Hanks,Bill Paxton,Kevin Bacon,Gary Sinise,269197,"173,837,933"


In [10]:
metadata = metas.to_dict('records')
for movie in metadata:
    movie['Actors'] =', '.join(movie['Actors'])
metadata


[{'Released_Year': '1994',
  'Runtime': '142 min',
  'Genre': 'Drama',
  'Meta_score': 80.0,
  'Director': 'Frank Darabont',
  'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler'},
 {'Released_Year': '1972',
  'Runtime': '175 min',
  'Genre': 'Crime, Drama',
  'Meta_score': 100.0,
  'Director': 'Francis Ford Coppola',
  'Actors': 'Marlon Brando, Al Pacino, James Caan, Diane Keaton'},
 {'Released_Year': '2008',
  'Runtime': '152 min',
  'Genre': 'Action, Crime, Drama',
  'Meta_score': 84.0,
  'Director': 'Christopher Nolan',
  'Actors': 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine'},
 {'Released_Year': '1974',
  'Runtime': '202 min',
  'Genre': 'Crime, Drama',
  'Meta_score': 90.0,
  'Director': 'Francis Ford Coppola',
  'Actors': 'Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton'},
 {'Released_Year': '1957',
  'Runtime': '96 min',
  'Genre': 'Crime, Drama',
  'Meta_score': 96.0,
  'Director': 'Sidney Lumet',
  'Actors': 'Henry Fonda, Lee J. Cobb, Ma

In [12]:

for dictionary in metadata:
     if dictionary['Released_Year'] == 'PG':
         dictionary['Released_Year'] = 1994
     dictionary['Released_Year'] = int(dictionary['Released_Year'])
     dictionary['Runtime'] =  dictionary['Runtime'][:-4]
     dictionary['Runtime'] = int(dictionary['Runtime'])
metadata

TypeError: 'int' object is not subscriptable

In [13]:
metadata

[{'Released_Year': 1994,
  'Runtime': 142,
  'Genre': 'Drama',
  'Meta_score': 80.0,
  'Director': 'Frank Darabont',
  'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler'},
 {'Released_Year': 1972,
  'Runtime': 175,
  'Genre': 'Crime, Drama',
  'Meta_score': 100.0,
  'Director': 'Francis Ford Coppola',
  'Actors': 'Marlon Brando, Al Pacino, James Caan, Diane Keaton'},
 {'Released_Year': 2008,
  'Runtime': 152,
  'Genre': 'Action, Crime, Drama',
  'Meta_score': 84.0,
  'Director': 'Christopher Nolan',
  'Actors': 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine'},
 {'Released_Year': 1974,
  'Runtime': 202,
  'Genre': 'Crime, Drama',
  'Meta_score': 90.0,
  'Director': 'Francis Ford Coppola',
  'Actors': 'Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton'},
 {'Released_Year': 1957,
  'Runtime': 96,
  'Genre': 'Crime, Drama',
  'Meta_score': 96.0,
  'Director': 'Sidney Lumet',
  'Actors': 'Henry Fonda, Lee J. Cobb, Martin Balsam, John Fiedler'},
 {'Released

In [14]:
ids = [str(i) for i in range(1,1001)]
print(ids)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '

In [15]:
data = data_df['full info'].to_list()
data

['The Shawshank Redemption: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 "The Godfather: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.",
 'The Dark Knight: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.',
 'The Godfather: Part II: The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate.',
 '12 Angry Men: A jury holdout attempts to prevent a miscarriage of justice by forcing his colleagues to reconsider the evidence.',
 "The Lord of the Rings: The Return of the King: Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the

In [16]:
metadata

[{'Released_Year': 1994,
  'Runtime': 142,
  'Genre': 'Drama',
  'Meta_score': 80.0,
  'Director': 'Frank Darabont',
  'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler'},
 {'Released_Year': 1972,
  'Runtime': 175,
  'Genre': 'Crime, Drama',
  'Meta_score': 100.0,
  'Director': 'Francis Ford Coppola',
  'Actors': 'Marlon Brando, Al Pacino, James Caan, Diane Keaton'},
 {'Released_Year': 2008,
  'Runtime': 152,
  'Genre': 'Action, Crime, Drama',
  'Meta_score': 84.0,
  'Director': 'Christopher Nolan',
  'Actors': 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine'},
 {'Released_Year': 1974,
  'Runtime': 202,
  'Genre': 'Crime, Drama',
  'Meta_score': 90.0,
  'Director': 'Francis Ford Coppola',
  'Actors': 'Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton'},
 {'Released_Year': 1957,
  'Runtime': 96,
  'Genre': 'Crime, Drama',
  'Meta_score': 96.0,
  'Director': 'Sidney Lumet',
  'Actors': 'Henry Fonda, Lee J. Cobb, Martin Balsam, John Fiedler'},
 {'Released

In [17]:
new_collection.upsert(
    documents = data,
    metadatas= metadata,
    ids = ids
)

In [18]:
results=new_collection.query(
    query_texts="Genre: Action-Adventure,\
        Mood: fun, whimsical, \
        Era: Modern" ,
    n_results=5
)
print(results)
print(results['documents'][0])

{'ids': [['539', '876', '663', '191', '255']], 'distances': [[1.052819013595581, 1.0753287076950073, 1.2293051481246948, 1.2352608442306519, 1.236794114112854]], 'metadatas': [[{'Actors': 'Magali Noël, Bruno Zanin, Pupella Maggio, Armando Brancia', 'Director': 'Federico Fellini', 'Genre': 'Comedy, Drama, Family', 'Released_Year': 1973, 'Runtime': 123}, {'Actors': 'Samuel Armstrong, Ford Beebe Jr., Norman Ferguson, David Hand', 'Director': 'James Algar', 'Genre': 'Animation, Family, Fantasy', 'Meta_score': 96.0, 'Released_Year': 1940, 'Runtime': 125}, {'Actors': 'David Thewlis, Lesley Sharp, Katrin Cartlidge, Greg Cruttwell', 'Director': 'Mike Leigh', 'Genre': 'Comedy, Drama', 'Meta_score': 84.0, 'Released_Year': 1993, 'Runtime': 132}, {'Actors': 'Bette Davis, Anne Baxter, George Sanders, Celeste Holm', 'Director': 'Joseph L. Mankiewicz', 'Genre': 'Drama', 'Meta_score': 98.0, 'Released_Year': 1950, 'Runtime': 138}, {'Actors': 'Ewan McGregor, Ewen Bremner, Jonny Lee Miller, Kevin McKidd'

In [19]:
from transformers import pipeline
def eBERT_AI_v3(question):
    results=new_collection.query(query_texts=question,n_results=4)

    model_name_qa = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"

    nlp = pipeline('question-answering',  model=model_name_qa, tokenizer=model_name_qa)
    QA_input = {
    'question': question,
    'context': results['documents'][0][0]
    }
    res = nlp(QA_input)
    print(res)
    print(results['documents'][0][0])

In [20]:
#genres, mood, era, cast
query = "Genre: Action-Adventure,Mood: fun, whimsical , Era:No preference , Cast: No preference "
eBERT_AI_v3(query)

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/jacobzimmerman/anaconda3/envs/project_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to

{'score': 0.027076933532953262, 'start': 0, 'end': 13, 'answer': 'All About Eve'}
All About Eve: A seemingly timid but secretly ruthless ingénue insinuates herself into the lives of an aging Broadway star and her circle of theater friends.


In [21]:
from transformers import pipeline
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
def eBERT_AI_v4(question,num_recs,min_time=80,max_time=500,era_start=1900,era_end=2024):
    results=new_collection.query(query_texts=question,
    n_results=num_recs,
    where={"$and":[{"Released_Year":{"$lte":era_end}},{"Released_Year":{"$gte":era_start}},{"Runtime":{"$lte":max_time}},{"Runtime":{"$gte":min_time}},]}
    )

    movie_results =""
    model_name_qa = "deepset/roberta-base-squad2"
    for i in range(num_recs):
        nlp = pipeline('question-answering',  model=model_name_qa, tokenizer=model_name_qa)
        QA_input = {
        'question': 'Recommend me a movie',
        'context': results['documents'][0][i]
        }
        res = nlp(QA_input)
        print(res)
        movie_results += res['answer'] + ','


    print(results['documents'][0])
    print(movie_results[:-1])

In [22]:
#genres, mood, era, cast
query = "Two violent young men take a mother"
eBERT_AI_v4(query,4)

{'score': 2.407084878086607e-07, 'start': 0, 'end': 11, 'answer': 'Funny Games'}
{'score': 3.46688864283351e-07, 'start': 85, 'end': 142, 'answer': 'mysterious neighbor inserts herself into their household.'}
{'score': 9.226098336512223e-05, 'start': 0, 'end': 10, 'answer': 'Badhaai ho'}
{'score': 0.0003302571130916476, 'start': 0, 'end': 17, 'answer': 'Road to Perdition'}
['Funny Games: Two violent young men take a mother, father, and son hostage in their vacation cabin and force them to play sadistic "games" with one another for their own amusement.', 'Mommy: A widowed single mother, raising her violent son alone, finds new hope when a mysterious neighbor inserts herself into their household.', 'Badhaai ho: A man is embarrassed when he finds out his mother is pregnant.', "Road to Perdition: A mob enforcer's son witnesses a murder, forcing him and his father to take to the road, and his father down a path of redemption and revenge."]
Funny Games,mysterious neighbor inserts herself int

In [23]:
results=new_collection.query(
    query_texts='Animated Sci-fi adventure that takes place in space',
    where={"$and":[{"Released_Year":{"$gte":1977}},{"Runtime":{"$lte":500}},{"Runtime":{"$gte":80}},]},
    n_results=5
)
print(results)
print(results['documents'][0])

{'ids': [['390', '22', '624', '67', '652']], 'distances': [[1.1217106580734253, 1.1898033618927002, 1.2370418310165405, 1.2624448537826538, 1.272426962852478]], 'metadatas': [[{'Actors': 'Eli Marienthal, Harry Connick Jr., Jennifer Aniston, Vin Diesel', 'Director': 'Brad Bird', 'Genre': 'Animation, Action, Adventure', 'Meta_score': 85.0, 'Released_Year': 1999, 'Runtime': 86}, {'Actors': 'Matthew McConaughey, Anne Hathaway, Jessica Chastain, Mackenzie Foy', 'Director': 'Christopher Nolan', 'Genre': 'Adventure, Drama, Sci-Fi', 'Meta_score': 74.0, 'Released_Year': 2014, 'Runtime': 169}, {'Actors': 'Sam Worthington, Zoe Saldana, Sigourney Weaver, Michelle Rodriguez', 'Director': 'James Cameron', 'Genre': 'Action, Adventure, Fantasy', 'Meta_score': 83.0, 'Released_Year': 2009, 'Runtime': 162}, {'Actors': 'Ben Burtt, Elissa Knight, Jeff Garlin, Fred Willard', 'Director': 'Andrew Stanton', 'Genre': 'Animation, Adventure, Family', 'Meta_score': 95.0, 'Released_Year': 2008, 'Runtime': 98}, {'Ac

In [24]:
!pip install pytube
from pytube import Search
import logging

pytube_logger = logging.getLogger('pytube')
pytube_logger.setLevel(logging.ERROR)
def search_youtube(query):
    search = Search(query)
    results = search.results

    for video in results:
        if 'Trailer'  not in video.title:
            print(f'Title: {video.title}, URL: {video.watch_url}')

search_youtube('Naruto clips')


Title: Miss The Rage x Naruto’s First Rasengan, URL: https://youtube.com/watch?v=nvMzU9BZpDk
Title: Uzumaki Naruto Raw Clips For Editing, URL: https://youtube.com/watch?v=dURPeblimx8
Title: Hatake Kakashi and Team 10 vs Kakuzu and Hidan | Naruto | - [English Dub], URL: https://youtube.com/watch?v=hn7gY7NXTFA
Title: Naruto and Sasuke vs Madara Uchiha - Naruto Sasuke Gets Sage Of Six Paths Power, URL: https://youtube.com/watch?v=tlfUk4MDqZc
Title: Naruto vs Neji, Naruto uses the nine tail fox chakra and beats Neji on the chunin exams, full fight, URL: https://youtube.com/watch?v=746ExUjRfhg
Title: Naruto Vs Pain English Dub - HD, URL: https://youtube.com/watch?v=Pj9c5F7lKzw
Title: Naruto SCARES THE S**T Out of Shin, URL: https://youtube.com/watch?v=6P-MZYrfjPE
Title: Naruto vs Kakashi Rasengan vs Rasenshuriken, URL: https://youtube.com/watch?v=zf6LCSzKNCI
Title: Naruto vs Sasuke Naruto Shippuden Finale Caught in 4K Part 1, URL: https://youtube.com/watch?v=m2cwistWD7Q
Title: Naruto's Firs